In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
from datetime import datetime

SYMBOL = "AAPL"
INTERVAL = "1m"
PERIOD = "1d"
SHORT_WINDOW = 1
LONG_WINDOW = 2
INITIAL_BALANCE = 100000
TRADE_QUANTITY = 10

balance = INITIAL_BALANCE
position = 0
entry_price = 0

def fetch_data():
    df = yf.download(
        tickers=SYMBOL,
        period=PERIOD,
        interval=INTERVAL,
        auto_adjust=False,
        progress=False
    )
    return df.dropna()

def apply_strategy(df):
    df["SMA_Short"] = df["Close"].rolling(SHORT_WINDOW).mean()
    df["SMA_Long"] = df["Close"].rolling(LONG_WINDOW).mean()
    return df

def trade(df):
    global balance, position, entry_price

    price = df["Close"].iloc[-1].item()
    short_ma = df["SMA_Short"].iloc[-1]
    long_ma = df["SMA_Long"].iloc[-1]

    if np.isnan(short_ma) or np.isnan(long_ma):
        return

    if short_ma > long_ma and position == 0:
        cost = price * TRADE_QUANTITY
        if balance >= cost:
            balance -= cost
            position = 1
            entry_price = price
            print(f"[{datetime.now()}] BUY {TRADE_QUANTITY} @ {price:.2f}")
            print(f"Balance: {balance:.2f}")

    elif short_ma < long_ma and position == 1:
        balance += price * TRADE_QUANTITY
        profit = (price - entry_price) * TRADE_QUANTITY
        position = 0
        print(f"[{datetime.now()}] SELL {TRADE_QUANTITY} @ {price:.2f}")
        print(f"Profit: {profit:.2f}")
        print(f"Balance: {balance:.2f}")

def run_bot():
    print("Trading Bot Started")
    print(f"Symbol: {SYMBOL}")
    print(f"Initial Balance: {INITIAL_BALANCE}")

    while True:
        try:
            df = fetch_data()
            df = apply_strategy(df)
            trade(df)
            time.sleep(60)
        except Exception as e:
            print("Error:", e)
            time.sleep(60)

if __name__ == "__main__":
    run_bot()


Trading Bot Started
Symbol: AAPL
Initial Balance: 100000
[2025-12-26 20:46:49.563939] BUY 10 @ 274.38
Balance: 97256.20
